# EOF Analysis of AR days

* Multivariate EOF analysis of H, U and V at 250 hPa in Extratropics and Q, U, and V at 850 hPa in Tropics

In [ ]:
# Import Python modules
import os, sys
from pathlib import Path
import numpy as np
import numpy.ma as ma
import pandas as  pd
import xarray as xr
from sklearn.cluster import KMeans
from itertools import cycle
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.colors import ListedColormap
from matplotlib import rcParams
import matplotlib.ticker as mticker
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

from IPython.display import Image, display

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from timeseries import persistence
from eofs import *
from ar_funcs import preprocess_ar_area_subregions
from kmeans import *

In [ ]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                            # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/cEOF_tropics_extratropics/'      # figures


In [ ]:
# Set a default font for all matplotlib text (can only set this ONCE; must restart kernel to change it)

rcParams['font.family'] = 'sans-serif'   # set the default font family to 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'    # set the default sans-serif font to 'Arial'

## Data

### AR time series

In [ ]:
## Select MERRA2 or ERA5
reanalysis = 'era5'

if reanalysis == 'era5':
    start_date = '1979-01-01'
    end_date = '2018-12-31'
    filename = 'ar_catalog_ERAI_fraction_HASIAsubregions.nc'
## if MERRA2
else:
    start_date = '1980-01-01'
    end_date = '2017-12-31'
    filename = 'ar_catalog_fraction_HASIAsubregions.nc'
    
f1 = path_to_data + 'CH1_generated_data/' + filename
ds = xr.open_dataset(f1)
# Set dates
ds = ds.sel(time=slice(start_date, end_date))
## Preprocess AR subregions - get dataframe of AR days based on area threshold
df = preprocess_ar_area_subregions(df=ds.to_dataframe(), thres=0.3)
# Show table
df.head()

### Import reanalysis

#### ERA5

In [ ]:
## Set variable names (for saving data/figs)
var_names = 'HUV250QUV850'
eofmode = 't' # s or t
dispmat = 'cor' # dispersion matrix type correlation/covariance

# Select lat/lon grid 
# Tropics/Extratropics Domain
lonmin = 0
lonmax = 120
latmin = -15
latmax = 65

lev = [850., 250.]

# for figure names for testing different configurations
fname_id = var_names + eofmode + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + dispmat

def preprocess(ds):
    '''keep only selected lats and lons'''
    return ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax), level=lev)

# open HUV anomaly data
filepath_pattern = path_to_data + 'ERA5/huvq/anomalies/daily_filtered_anomalies_huvq_*.nc'
    
    
f2 = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, combine='by_coords')



In [ ]:
# Create new dataset to rename lat lon
ds = xr.Dataset({'H': (['time', 'level', 'lat', 'lon'], f2['z'].values),
                 'U': (['time', 'level',  'lat', 'lon'], f2['u'].values),
                 'V': (['time', 'level',  'lat', 'lon'], f2['v'].values),
                 'QV': (['time', 'level',  'lat', 'lon'], f2['q'].values)},
                      coords={'time': (['time'], f2.time.values),
                              'level': (['level'], f2.level.values),
                              'lat': (['lat'], f2.latitude.values),
                              'lon': (['lon'], f2.longitude.values)})

ds
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))

In [ ]:
# ## Set variable names (for saving data/figs)
# var_names = 'HUVQ500'
# eofmode = 't' # s or t
# dispmat = 'cor' # dispersion matrix type correlation/covariance

# ## Select lat/lon grid 
# # Tropics/Extratropics Domain
# lonmin = 0
# lonmax = 120
# latmin = -15
# latmax = 65

# # for figure names for testing different configurations
# fname_id = var_names + eofmode + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + dispmat

# ### MERRA2 DATA ###
# def preprocess(ds):
#     '''keep only selected lats and lons'''
#     return ds.sel(lat=slice(latmin, latmax), lon=slice(lonmin, lonmax))

# # open H data
# filepath_pattern = path_to_data + 'MERRA2/anomalies/H500/daily_*.nc'

# ds_h = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, concat_dim='time', combine='by_coords')
# print('ds size in GB {:0.2f}\n'.format(ds_h.nbytes / 1e9))

# # open QV data
# filepath_pattern = path_to_data + 'MERRA2/anomalies/QV500/daily_*.nc'

# ds_q = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, concat_dim='time', combine='by_coords')
# print('ds size in GB {:0.2f}\n'.format(ds_q.nbytes / 1e9))

# ## open UV data
# filepath_pattern = path_to_data + 'MERRA2/anomalies/UV500/daily_*.nc'
# ds_uv = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, combine='by_coords')
# print('ds size in GB {:0.2f}\n'.format(ds_uv.nbytes / 1e9))

# ## combine H and UV data into 1 ds object
# ds = xr.merge([ds_h, ds_uv.U, ds_uv.V, ds_q])
# # ds

In [ ]:
# Add AR time series to merra; set as coordinate variables
ds['ar'] = ('time', df.ar)
ds = ds.set_coords('ar')

ds['location'] = ('time', df.location)
ds = ds.set_coords('location')

# print dataset
print(ds)

### Data Subset Selection

In [ ]:
# Trim date range
if reanalysis == 'era5':
    start_date = '1979-12-01'
    end_date = '2018-02-28'
    ys = 1979
    ye = 2018
else:
    start_date = '1980-12-01'
    end_date = '2017-02-28'
    ys = 1980
    ye = 2017
    
idx = slice(start_date, end_date)
ds = ds.sel(time=idx)

# Select DJF months
idx = (ds.time.dt.month >= 12) | (ds.time.dt.month <= 2)
ds = ds.sel(time=idx)

# # Select AR days JUST IN R01
# idx = (ds.ar >= 1) & (ds.location == 'R01')
# Select AR days in all subregions
idx = (ds.ar >= 1)
ds_ar = ds.sel(time=idx)
# ds_ar = ds

# print results
print(ds_ar)

In [ ]:
# Count number of independent AR events

years = np.arange(ys, ye) 
nyrs = len(years)
total_events = 0
for k in range(nyrs-1):    
    # Extract single DJF season
    date1 = "{}-12-01".format(years[k])
    date2 = "{}-02-28".format(years[k+1])
    x = ds.ar.sel(time=slice(date1,date2)).values
    # Count AR events in that season
    tags, tmp = persistence(x)
    # Add to running event count
    total_events += tmp

print("Number of independent AR events: ", total_events)

## Preprocessing

### Reshape, center, and standardize data matrix

In [ ]:
%%time
# Load merra_ar dataset into memory
ds_ar = ds_ar.load()


In [ ]:
# "Tropics" Domain
tlonmin = 0
tlonmax = 120
tlatmin = -15
tlatmax =  25

# "Extratropics" Domain
etlonmin = 0
etlonmax = 120
etlatmin = 25
etlatmax =  65

## Create list of variable arrays
## FOR MERRA2 need to switch back to slice(latmin, latmax)
# Extratropic variables
var1 = ds_ar.U.sel(lon=slice(etlonmin, etlonmax), lat=slice(etlatmax, etlatmin), level=250.)
var2 = ds_ar.V.sel(lon=slice(etlonmin, etlonmax), lat=slice(etlatmax, etlatmin), level=250.)
var3 = ds_ar.H.sel(lon=slice(etlonmin, etlonmax), lat=slice(etlatmax, etlatmin), level=250.)

# Tropics variables
var4 = ds_ar.U.sel(lon=slice(tlonmin, tlonmax), lat=slice(tlatmax, tlatmin), level=850.)
var5 = ds_ar.V.sel(lon=slice(tlonmin, tlonmax), lat=slice(tlatmax, tlatmin), level=850.)
var6 = ds_ar.QV.sel(lon=slice(tlonmin, tlonmax), lat=slice(tlatmax, tlatmin), level=850.)

var_list = [var1, var2, var3, var4, var5, var6]

# Check that sizes of arrays match
for i, in_array in enumerate(var_list):
    # Extract variable as numpy array
    var1 = in_array.values
    print(var1.shape)


In [ ]:
%%time
# Weight the data by the square root of the cosine of the lat
var_list = spatial_weights(var_list)

## Flatten data to [time x space]
var_list = flatten_array(var_list)

## Center data
var_list = center_data(var_list)

## Standardize Arrays by removing the mean and dividing by the standard deviation of the columns
## For multivariate, place into single flattened array
Xs = standardize_arrays(var_list, mode=eofmode, dispersion_matrix=dispmat)


## EOF Analysis

In [ ]:
%%time

# Compute eigenvalues & eigenvectors
R, evals, evecs = calc_eigs(z=Xs, mode=eofmode)

print('Eigenvalues: ', evals.shape)
print(evals, '\n')

print('Eigenvectors: ', evecs.shape)
print(np.round(evecs, 3), '\n')

In [ ]:
## Plot dispersion matrix
plt.contourf(R)
plt.colorbar()
title = 'Dispersion Matrix (' + dispmat + ')'
plt.title(title)

### Explained Variance

In [ ]:
# Calculate the percent explained var by each eigenvector
pctvar = pct_variance(evals)

# Number of EOFs that explain more than 1% of the total variance
idx = pctvar[pctvar >= 1.0]
neofs = len(idx)

# print exp var >= 1.0
cumvar = np.sum(pctvar[0:neofs-1])
print(f'Cumulative variance explained by the first {neofs} EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var: neofs = 4
cumvar = np.sum(pctvar[0:3])
print(f'Cumulative variance explained by the first 4 EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var for 4 eofs
for k in range(4):
    print(f'{k+1} \t {pctvar[k]:.2f}%')

### North Test

In [ ]:
err = north_test(evals, total_events)
upper = pctvar + err
lower = pctvar - err

print(np.round(upper[0:6],3))
print(np.round(pctvar[0:6],3))
print(np.round(lower[0:6],3))

### Fig 2: Variance

In [ ]:
# set seaborn style
sns.set()
sns.set_style("ticks", {'patch.force_edgecolor':False})

# create figure
fig, ax = plt.subplots(figsize=(6,4))

# plot data
xvals = np.arange(neofs) + 1
ax.bar(xvals, pctvar[0:neofs], yerr=err[0:neofs], 
       color='tab:blue', alpha=0.8)

# x-axis
ax.set_xlabel('EOF')
ax.set_xticks(xvals)

# y-axis
ax.set_ylabel('Explained Variance (%)')
yticks = np.arange(0,16,3)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks) 

# save fig
filepath = path_to_figs + 'exp_variance_' + fname_id + '.png'
plt.savefig(filepath, dpi=300)

# show
plt.show()

### EOFs and PCs

In [ ]:
# Calculate EOFS (spatial modes)
neofs = 4
eofs = calc_eofs(Xs, evecs, evals, neofs, mode=eofmode)

# Split eofs into separate arrays for each variable
ntim, nlat, nlon = var2.shape
npts = nlat*nlon
nvar = len(var_list)
# Reshape spatial dim back to 2D map
eofmodes = var_list
for i in np.arange(len(var_list)):
    tmp = eofs[:,i*npts:(i+1)*npts]
    eofmodes[i] = np.reshape(tmp, (neofs,nlat,nlon))

In [ ]:
# Calculate PCs (time coefficients)
npcs = 4
pcs = calc_pcs(Xs, evecs, evals, npcs, mode=eofmode)
# results in [ntim, npcs] to plot in PC plot

### Save Eigenvectors, eigenvalues, and temporal loadings.

### Loadings

In [ ]:
## in the case of t-mode these are our "pcs" or time-coefficients
## in the case of s-mode, these are our "eofs" or spatial loadings
loads = pcs

In [ ]:
# Save eigenvalues, eigenvectors, and loadings

neofs = 4   # number of EOFs to save (evecs, loadings3)

outfile = path_to_out + 'eigenvalues_'+ fname_id + '.txt'
np.savetxt(outfile, evals, fmt='%.5f')

outfile = path_to_out + 'eigenvectors_'+ fname_id + '.txt'
np.savetxt(outfile, evecs[:,0:neofs], fmt='%.5f', delimiter=',')

outfile = path_to_out + 'loadings_'+ fname_id + '.txt'
np.savetxt(outfile, loads[:,0:neofs], fmt='%.4f', delimiter=',')


### Fig 3: Spatial Modes

In [ ]:
# Panel Plot of Spatial Modes

# Data for plotting extratropics
lons_et = var2.lon.data
lats_et = var2.lat.data
udat_et = eofmodes[0][0:neofs,:,:]
vdat_et = eofmodes[1][0:neofs,:,:]
data_et = eofmodes[2][0:neofs,:,:]

# Data for plotting tropics
lons_t = var4.lon.data
lats_t = var4.lat.data
udat_t = eofmodes[3][0:neofs,:,:]
vdat_t = eofmodes[4][0:neofs,:,:]
data_t = eofmodes[5][0:neofs,:,:]

print(np.min(data_et), np.max(data_et))
print(np.min(data_t), np.max(data_t))

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

# subtitles
eof_label = [ ]
pc_label = [ ]
var_label = [ ]
for k in range(neofs):
    eof_label.append("EOF{:1d}".format(k+1,))
    pc_label.append("PC{:1d}".format(k+1,))
    var_label.append("{:.2f}%".format(pctvar[k]))

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,11))
filepath = path_to_figs + 'eofs_'+ fname_id + '.png'
nrows = 2
ncols = 2
# sns.set_style('ticks')

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 
                111, 
                axes_class=axes_class,
                nrows_ncols=(nrows, ncols), 
                axes_pad = 0.55,
                cbar_location='bottom', 
                cbar_mode='single',
                cbar_pad=0.0, 
                cbar_size='2.5%',
                label_mode='')

#newcmap = cmocean.tools.crop_by_percent(cmo.matter, 15, which='max', N=None)

# Loop for drawing each plot
for k, ax in enumerate(axgr):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
#     ax = draw_basemap(ax, extent=None, xticks=dx, yticks=dy)
    
    # Add contour fill plot for extratropics
#     clevs = np.arange(-15,16,2)
    clevs = np.arange(-50,55,2)
    cf = ax.contourf(lons_et, lats_et, data_et[k,:,:], transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
#     # add vectors for extratropics
#     ax.quiver(lons_et, lats_et, udat_et[k,:,:], vdat_et[k,:,:], transform=datacrs,
#               color='black', pivot='middle', regrid_shape=30) 
    
    # Add contour fill plot for q500
    cf = ax.contourf(lons_t, lats_t, data_t[k,:,:], transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
#     # add vectors for tropics
#     ax.quiver(lons_t, lats_t, udat_t[k,:,:], vdat_t[k,:,:], transform=datacrs,
#               color='black', pivot='middle', regrid_shape=30)
    
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k], loc='right', fontsize=12)
    
# single colorbar
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('m', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Display figure
plt.savefig(filepath, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(9,11))
fig.dpi = 200
fname = path_to_figs + 'pc_'+ fname_id
fmt = 'png'
results = loads[:,:4]

X, nplots = results.shape
x = np.arange(len(results))

for i in np.arange(nplots):
    ax = plt.subplot(4, 1, i+1)
    ax.plot(x, results[:,i], '-')
    ax.axhline(0, color='k')
    ax.set_ylim(-1, 1)
#     ax.set_ylabel('Normalized Units')
    # subtitles
    ax.set_title(pc_label[i], loc='left', fontsize=12)
    ax.set_title(var_label[i], loc='right', fontsize=12)

plt.subplots_adjust(hspace=0.35, wspace=0.003)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()


plotFile = fname + '.png'
print(plotFile)
display(Image(plotFile))

### EOF represented as correlation

In [ ]:
# put PCS into dataset object
npcs = 2
ds_pc = xr.Dataset({'pc': (['time', 'pcs'], loads[:,0:npcs])},
                      coords={'time': (['time'], ds_ar.time.values),
                              'pcs': (['pcs'], np.arange(npcs))})
ds_pc.pc.sel(pcs=0)

# select the pc to correlate
pc1 = ds_pc.pc.sel(pcs=0)
pc2 = ds_pc.pc.sel(pcs=1)
# calculate statistics between AR ds and PCs
cor1, pval1, tstat1 = correlation_pvalue(x=pc1, y=ds_ar, lagx=0, lagy=0, n=total_events)
cor2, pval2, tstat2 = correlation_pvalue(x=pc2, y=ds_ar, lagx=0, lagy=0, n=total_events)

# ## combine cor1 and cor2 into 1 ds object
ds_cor = xr.concat([cor1, cor2], dim='pcs')
ds_pval = xr.concat([pval1, pval2], dim='pcs')
ds_tstat = xr.concat([tstat1, tstat2], dim='pcs')

In [ ]:
# Panel Plot of Correlation with EOF1 and EOF2

# Data for plotting extratropics
lons = ds_ar.lon.data
lats = ds_ar.lat.data

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

nplots=8

# subtitles
eof_label = [ ]
pool = cycle(['1', '2'])
pool2 = cycle([0, 1])
for k in range(nplots):
    neof = next(pool)
    eof_label.append("EOF{0}".format(neof))
    
var_label = ['H', 'H', 'QV', 'QV', 'V', 'V', 'U', 'U']    


#### Plot 250 hPa correlations

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,11))
filepath = path_to_figs + 'cors_huvq250'+ fname_id + '.png'
nrows = 4
ncols = 2
# sns.set_style('ticks')

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 
                111, 
                axes_class=axes_class,
                nrows_ncols=(nrows, ncols), 
                axes_pad = 0.55,
                cbar_location='bottom', 
                cbar_mode='single',
                cbar_pad=0.0, 
                cbar_size='2.5%',
                label_mode='')

# Loop for drawing each plot
for k, ax in enumerate(axgr):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
    # variable to plot
    var_plot = var_label[k]
    neof = next(pool2)
#     data = ds_cor[var_plot].sel(level=250., pcs=neof).where(np.abs(ds_pval[var_plot].sel(level=250., pcs=neof))<=0.05)
    data = ds_cor[var_plot].sel(level=250., pcs=neof).where(np.abs(ds_tstat[var_plot].sel(level=250., pcs=neof))>=1.96)
#     data = ds_cor[var_plot].sel(level=250., pcs=neof)
    # Add contour fill plot for correlation maps
    clevs = np.arange(-1,1.1,0.1)
    cf = ax.contourf(lons, lats, data, transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
    
#     # Add contours for pval
#     kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%0.2f',
#                   'rightside_up': True, 'use_clabeltext': True}
#     clevs = np.arange(0.05, 2.25, 0.25)
#     pvaldata = ds_pval[var_plot].sel(level=250., pcs=neof)
#     cs = ax.contour(lons, lats, pvaldata, levels=clevs, colors='k',
#                         linewidths=1.0, linestyles='dashed', transform=datacrs)
    
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k]+' 250 hPa', loc='right', fontsize=12)
    
# single colorbar
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('correlation coefficient', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Display figure
plt.savefig(filepath, dpi=200, bbox_inches='tight')
plt.show()

#### Plot 850 hPa correlations

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,11))
filepath = path_to_figs + 'cors_huvq850'+ fname_id + '.png'
nrows = 4
ncols = 2
# sns.set_style('ticks')

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 
                111, 
                axes_class=axes_class,
                nrows_ncols=(nrows, ncols), 
                axes_pad = 0.55,
                cbar_location='bottom', 
                cbar_mode='single',
                cbar_pad=0.0, 
                cbar_size='2.5%',
                label_mode='')

# Loop for drawing each plot
for k, ax in enumerate(axgr):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
    var_plot = var_label[k]
    neof = next(pool2)
#     data = ds_cor[var_plot].sel(level=850., pcs=neof).where(np.abs(ds_pval[var_plot].sel(level=850., pcs=neof))<=0.05)
    data = ds_cor[var_plot].sel(level=850., pcs=neof).where(np.abs(ds_tstat[var_plot].sel(level=850., pcs=neof))>=1.96)
#     data = ds_cor[var_plot].sel(level=850., pcs=neof)
    # Add contour fill plot for correlation maps
    clevs = np.arange(-1,1.1,0.1)
    cf = ax.contourf(lons, lats, data, transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
    
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k]+' 850 hPa', loc='right', fontsize=12)
    
# single colorbar
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('correlation coefficient', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Display figure
plt.savefig(filepath, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
### Scatterplot of PC1 v. PC2
x = loads[:, 0]
y = loads[:, 1]

plt.plot(x, y, 'o', color='black')
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.show()

### K-means clustering

In [ ]:
# Determine optimal K

# maximum number of clusters (number of iterations)
kmax =15
# number of eofs
neofs = 2
# input data
xdata = loads[:,0:neofs]

# Elbow plot
outfile = path_to_figs + 'elbow'+ fname_id
plot_optimal_k(xdata, kmax, filename=outfile)


In [ ]:
## K-means cluster analysis

# Number of clusters
nk = 2

# Input data
xdata = loads[:,0:neofs]

# Compute k means and assign each point to a cluster
kmeans = KMeans(n_clusters=nk)
kmeans.fit(xdata)
cluster = kmeans.predict(xdata)

# LLJ category labels (llj days only)
ar_cat = cluster + 1


In [ ]:
# Count number of days in each cluster
klabels, counts = np.unique(ar_cat, return_counts=True)

# Save counts to txt file
res = np.column_stack((klabels,counts))
headstr = 'AR_TYPE, COUNT'
outfile = path_to_out + fname_id + 'k_counts.txt'
np.savetxt(outfile, res, delimiter=',', fmt='%d', header=headstr)



In [ ]:
# Cluster centroids (nclust x neofs)
centroids = kmeans.cluster_centers_

# Save centroids to txt file
res = np.column_stack((klabels,centroids))
headstr = "AR_TYPE, EOF1, EOF2, EOF3, EOF4"
outfile = path_to_out + fname_id + 'centroids.txt'
np.savetxt(outfile, res, delimiter=',', fmt='%s', header=headstr)


In [ ]:
## Save AR location, loadings (EOF1-2), and category label (AR days only)

# Vector of AR dates
dates_arDays =  ds_ar.time.values

# Create new dataframe
data = {'LOC':ds_ar.location.values,
        'EOF1':loads[:,0],
        'EOF2':loads[:,1],
        'AR_CAT':ar_cat}
df_out = pd.DataFrame(data, index=dates_arDays)

## Create EOF "category" based on if exceeds std dev 
std_PC = np.std(df_out)
mean_PC = np.mean(df_out)

df_out['EOF_CAT1'] = 0
df_out['EOF_CAT2'] = 0
df_out['EOF_CAT'] = 0
idx = (df_out['EOF1']-mean_PC.EOF1 >= std_PC.EOF1) | (df_out['EOF1']-mean_PC.EOF1 <= std_PC.EOF1*-1)
df_out.loc[idx, 'EOF_CAT1'] = 1
df_out.loc[idx, 'EOF_CAT'] = 1

idx = (df_out['EOF2']-mean_PC.EOF2 >= std_PC.EOF2) | (df_out['EOF2']-mean_PC.EOF2 <= std_PC.EOF2*-1)
df_out.loc[idx, 'EOF_CAT2'] = 1
df_out.loc[idx, 'EOF_CAT'] = 2
print(df_out)
# Export dataframe as csv
outfile = path_to_out + fname_id + 'hma_AR-types-loadings.csv'
df_out.to_csv(outfile)
print(outfile)

In [ ]:
## Save time series of all DJF days with AR types

# Arrays with ALL DJF days
dates_allDays = ds.time.values
ar_cat_allDays = np.zeros(len(dates_allDays), dtype=int)
eof_cat_allDays = np.zeros(len(dates_allDays), dtype=int)
eof_cat1_allDays = np.zeros(len(dates_allDays), dtype=int)
eof_cat2_allDays = np.zeros(len(dates_allDays), dtype=int)

# Loop over ar days and match to ar_full 
for i, date in enumerate(dates_arDays):
    idx = np.where(dates_allDays == date)
    ar_cat_allDays[idx] = ar_cat[i]
    eof_cat_allDays[idx] = df_out.EOF_CAT.values[i]
    eof_cat1_allDays[idx] = df_out.EOF_CAT1.values[i]
    eof_cat2_allDays[idx] = df_out.EOF_CAT2.values[i]

# Create dataframe
data = {'AR_CAT':ar_cat_allDays,
        'EOF_CAT':eof_cat_allDays,
        'EOF_CAT1':eof_cat1_allDays,
        'EOF_CAT2':eof_cat2_allDays}
df_out = pd.DataFrame(data, index=dates_allDays)
print(df_out)

outfile = path_to_out + fname_id + 'hma_AR-types-djf.csv'
df_out.to_csv(outfile)


In [ ]:
print(outfile)